# Assignment 2 FIT5149 

### Group no. 45



## Libraries and dependancies

In [7]:
from __future__ import absolute_import, division, print_function, unicode_literals
#!pip install -q tensorflow-gpu==2.0.0-alpha0
#import tensorflow_datasets as tfds
import tensorflow as tf
import pandas as pd
import tokenize
import textblob
import nltk
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

## KERAS
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.core import Dense, SpatialDropout1D
from keras.layers.convolutional import Conv1D
from keras.layers.embeddings import Embedding
from keras.layers.pooling import GlobalMaxPooling1D
from keras.preprocessing.sequence import pad_sequences
from keras.utils import np_utils
from keras.models import load_model
import collections
nltk.download("wordnet")
from textblob import Word
import numpy as np
from keras.callbacks import EarlyStopping
from nltk.corpus import stopwords
pd.set_option('display.max_colwidth', -1) 
from sklearn import model_selection
import re
# Keras
from keras.preprocessing.text import Tokenizer
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
# Others
import nltk
import string
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.manifold import TSNE

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/akshitchawla/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Google Colab setup and drive mount

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

## Loading the datasets

In [8]:
df_train = pd.read_csv('/Users/akshitchawla/Documents/fit5149_new/Assignment 2/train_data.csv')
#df_train = pd.read_csv('gdrive/FIT5149_Assignment_2/Assignment2/Training Dataset-20190429/train_data.csv')
df_train_lb = pd.read_csv('/Users/akshitchawla/Documents/fit5149_new/Assignment 2/train_label.csv')
# combining the train data and label based on trn_id
df_full = df_train.merge(df_train_lb,on=['trn_id'], how='inner')
test_data = pd.read_csv('/Users/akshitchawla/Documents/fit5149_new/Assignment 2/test_data.csv')

# Exploratoring Data dimensions without preprocessing

In [9]:
df_full.shape # shape of the combined file

(650000, 3)

In [10]:
## memory usage and other stats
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 650000 entries, 0 to 649999
Data columns (total 3 columns):
trn_id    650000 non-null object
text      650000 non-null object
label     650000 non-null int64
dtypes: int64(1), object(2)
memory usage: 19.8+ MB


In [11]:
## checking the label distribution
df_full['label'].value_counts().sort_values(ascending=False)

1    130000
2    130000
3    130000
4    130000
5    130000
Name: label, dtype: int64

## Preprocessing the data

-- Sorry please scroll down

In [12]:
# contraction list declaration
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have",
  "aint": "am not",
  "arent": "are not",
  "cant": "cannot",
  "cantve": "cannot have",
  "cause": "because",
  "couldve": "could have",
  "couldnt": "could not",
  "couldntve": "could not have",
  "didnt": "did not",
  "doesnt": "does not",
  "dont": "do not",
  "hadnt": "had not",
  "hadntve": "had not have",
  "hasnt": "has not",
  "havent": "have not",
  "hed": "he would",
  "hedve": "he would have",
  "hell": "he will",
  "hellve": "he will have",
  "hes": "he is",
  "howd": "how did",
  "howdy": "how do you",
  "howll": "how will",
  "hows": "how is",
  "Id": "I would",
  "Idve": "I would have",
  "Ill": "I will",
  "Illve": "I will have",
  "Im": "I am",
  "Ive": "I have",
  "isnt": "is not",
  "itd": "it had",
  "itdve": "it would have",
  "itll": "it will",
  "itllve": "it will have",
  "its": "it is",
  "lets": "let us",
  "maam": "madam",
  "maynt": "may not",
  "mightve": "might have",
  "mightnt": "might not",
  "mightntve": "might not have",
  "mustve": "must have",
  "mustnt": "must not",
  "mustntve": "must not have",
  "neednt": "need not",
  "needntve": "need not have",
  "oclock": "of the clock",
  "oughtnt": "ought not",
  "oughtntve": "ought not have",
  "shant": "shall not",
  "shant": "shall not",
  "shantve": "shall not have",
  "shed": "she would",
  "shedve": "she would have",
  "shell": "she will",
  "shellve": "she will have",
  "shes": "she is",
  "shouldve": "should have",
  "shouldnt": "should not",
  "shouldntve": "should not have",
  "sove": "so have",
  "sos": "so is",
  "thatd": "that would",
  "thatdve": "that would have",
  "thats": "that is",
  "thered": "there had",
  "theredve": "there would have",
  "theres": "there is",
  "theyd": "they would",
  "theydve": "they would have",
  "theyll": "they will",
  "theyllve": "they will have",
  "theyre": "they are",
  "theyve": "they have",
  "tove": "to have",
  "wasnt": "was not",
  "wed": "we had",
  "wedve": "we would have",
  "well": "we will",
  "wellve": "we will have",
  "were": "we are",
  "weve": "we have",
  "werent": "were not",
  "whatll": "what will",
  "whatllve": "what will have",
  "whatre": "what are",
  "whats": "what is",
  "whatve": "what have",
  "whens": "when is",
  "whenve": "when have",
  "whered": "where did",
  "wheres": "where is",
  "whereve": "where have",
  "wholl": "who will",
  "whollve": "who will have",
  "whos": "who is",
  "whove": "who have",
  "whys": "why is",
  "whyve": "why have",
  "willve": "will have",
  "wont": "will not",
  "wontve": "will not have",
  "wouldve": "would have",
  "wouldnt": "would not",
  "wouldntve": "would not have",
  "yall": "you all",
  "yalls": "you alls",
  "yalld": "you all would",
  "yalldve": "you all would have",
  "yallre": "you all are",
  "yallve": "you all have",
  "youd": "you had",
  "youdve": "you would have",
  "youll": "you you will",
  "youllve": "you you will have",
  "youre": "you are",
  "youve": "you have"
}

c_re = re.compile('(%s)' % '|'.join(cList.keys()))

In [13]:
def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)
def replace_emoticon_happy(text):
    pattern = re.compile(r'(?::|;|=)(?:-)?(?:\)|D|P)')
    return pattern.sub(r'happy', text)
def replace_emoticon_sad(text):
    pattern = re.compile(r'([:><].?-?[@><cC(\[{\|]\|?|[D][:8;=X]<?|v.v|:/|:\\|:-\\|:-/)')
    return pattern.sub(r'sad', text)
def reduce_length(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)


In [14]:
## manually generated list of stop words so that negation words are not removed when training the model
STOPWORDS = ["before","more","ll","about","you'd","because","from","we","you'll","through",
 "were","up","re","all","do","your","where","am","won","mightn","it's","it","when",
 "yourself","that","above","y","of","than","by","haven","between","over","has","hers",
 "itself","too","most","she's","once","ve","few","off","that'll","his","m","as","how",
 "are","have","she","there","ours","doing","for","such","and","ma","again","own","will",
 "ourselves","whom","its","you've","an","they","you're","while","some","these","so","on",
 "themselves","if","a","shan","is","this","very","their","now","at","then","t","or","each","d","i",
 "into","which","had","our","did","him","was","what","yours","be","does","herself","me","those","them",
 "s","yourselves","here","to","myself","both","weren","himself","you","been","the","during","but","o",
 "in","other","who","after","her","why",
 "being","he","having","can","out","just","with","my","further","any","until","theirs","same"]

In [15]:
#nltk.download('stopwords')
# reset the index
df_full = df_full.reset_index(drop=True)
## regex to remove the special characters
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
## regex to remove other special or bad symbols
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
#STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    # replacing the happy emoticons with the word happy
    text = replace_emoticon_happy(text)
    # replacing the sad emoticons with sad word
    text = replace_emoticon_sad(text)
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = expandContractions(text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    text = reduce_length(text)
    text = ' '.join([Word(word).lemmatize() for word in text.split()])
    return text
#%%time

- The below code calls the above function for cleaning and preprocessing the text...takes about 8 min...

In [16]:
%%time
df_full['text'] = df_full['text'].apply(clean_text)

CPU times: user 9min 1s, sys: 1.68 s, total: 9min 3s
Wall time: 9min 5s


# EDA after Preprocessing

In [17]:
df_full['totalwords'] = df_full['text'].str.split().str.len()

In [18]:
max(df_full['totalwords'])

646

In [19]:
len(df_full['totalwords'])

650000

In [20]:
sum(df_full['totalwords'])/len(df_full['totalwords'])

64.68212

In [21]:
df_full['totalwords'] = df_full['text'].str.split().str.len()

print("lexical_diversity")
sum(df_full['totalwords'])/len(df_full['totalwords'])

lexical_diversity


64.68212

## Setting the model paramters

In [22]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 10000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

### tokenize the given data

In [23]:
tokenizer = Tokenizer(num_words=10000, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df_full['text'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 529317 unique tokens.


## Train data and train label padding

In [24]:
X = tokenizer.texts_to_sequences(df_full['text'].values)
X = pad_sequences(X, maxlen=250)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (650000, 250)


In [25]:
Y = pd.get_dummies(df_full['label']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (650000, 5)


In [26]:
X.shape[1]

250

In [27]:
Y[:5]

array([[0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [0, 0, 0, 0, 1]], dtype=uint8)

## Initialising the model

In [28]:
model = Sequential() 
## add the embedding layer to the model
model.add(Embedding(MAX_NB_WORDS, 300, input_length=X.shape[1]))
## initialise drop out parameter
model.add(Dropout(0.4))
# initialising the first layer of LSTM
model.add(LSTM(250, return_sequences=True,dropout=0.3, recurrent_dropout=0.3))
# initialising the second layer of LSTM
model.add(LSTM(250, dropout=0.2, recurrent_dropout=0.2))
# initialising the output layer
model.add(Dense(5, activation='softmax'))
## compile the model for loss fucntion optimizer and metrics
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

#check summary of the model
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 300)          3000000   
_________________________________________________________________
dropout_1 (Dropout)          (None, 250, 300)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 250, 250)          551000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 250)               501000    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 1255      
Total params: 4,053,255
Trainable params: 4,053,255
Non-trainable params: 0
_________________________________________________________________


## Running the model

- The below error is because the model was stopped midway.

In [29]:
epochs = 6
batch_size = 400
history = model.fit(X, Y, epochs=epochs, batch_size=batch_size,validation_split=0.15)

Train on 552500 samples, validate on 97500 samples
Epoch 1/6


KeyboardInterrupt: 

In [0]:
# saving the model for future production
model.save('final_model.h5')

## Pre processing the test data for prediction

In [0]:
#new_complaint = test_data['text']
test_data['text'] = test_data['text'].apply(clean_text)
test_data['text'] = test_data['text'].str.replace('\d+', '')
new_complaint = test_data['text']
seq = tokenizer.texts_to_sequences(new_complaint)


## Padding the sequence and predicting the labels

In [113]:
padded = pad_sequences(seq, maxlen = 250)
pred = model.predict(padded)
print(pred)

[[2.1780254e-01 3.6738023e-01 3.1277019e-01 8.8856846e-02 1.3190233e-02]
 [9.9011930e-05 3.0360518e-03 3.1540257e-01 6.4054340e-01 4.0918931e-02]
 [7.7738792e-01 2.1349868e-01 8.6008180e-03 3.8542200e-04 1.2716002e-04]
 ...
 [2.6421753e-01 6.1859310e-01 1.1584746e-01 1.2135351e-03 1.2841354e-04]
 [1.6019880e-04 1.0199553e-04 1.2862789e-04 4.7241622e-03 9.9488503e-01]
 [1.8277639e-03 1.1277750e-01 7.7003568e-01 1.1249510e-01 2.8639394e-03]]


### Getting the class label for test data using the highest probability

In [0]:
predicted = np.argmax(pred, axis=1)
## add one to all as python index starts from 0
new_list = [x+1 for x in predicted]

In [0]:
# setting the predicted dataframe
predictions = pd.DataFrame(data=new_list, columns=['label'])

In [0]:
# getting the labels from test data
predictions['test_id'] = test_data['test_id']

In [0]:
# write the csv file to disk/drive
predictions.to_csv('gdrive/My Drive/FIT5149_Assignment_2/Assignment2/predicted_final.csv', index = False)

In [0]:
padd